In [4]:
import numpy as np
import pandas as pd

In [5]:
train_df=pd.read_csv('Training_data.csv')
test_df=pd.read_csv('Testing_data.csv')
train_df.head()

,amt,is_fraud,user_id,latitudinal_distance,longitudinal_distance
0,122.86,0,6,0.488,0.719
1,113.40,0,29,0.136,0.393
2,89.35,0,28,0.031,0.910
3,61.00,0,15,0.492,0.134
4,98.24,0,12,0.522,0.371


In [6]:
# Assuming your DataFrame is called train_df
train_df = train_df[train_df['user_id'] == 1]
test_df = test_df[test_df['user_id'] == 1]
# Display the filtered DataFrame
print(train_df.head())

       amt  is_fraud  user_id  latitudinal_distance  longitudinal_distance
149  94.20         0        1                 0.771                  0.295
353  83.80         0        1                 0.942                  0.831
422  90.72         0        1                 0.767                  0.806
423  89.32         0        1                 0.154                  0.661
433  38.25         0        1                 0.981                  0.514


In [7]:
X_train = train_df.drop('is_fraud', axis=1)
y_train = train_df['is_fraud']

In [8]:
X_test = test_df.drop('is_fraud', axis=1)
y_test = test_df['is_fraud']

In [9]:
# print("Shape of X_train:", X_train.shape)
# print("Shape of X_test:", X_test.shape)
# print("Columns in X_train:", X_train.columns)
# print("Columns in X_test:", X_test.columns)
# try:
#     # The line that causes KeyError
#     print(X_train['user_id'])
# except KeyError as e:
#     print("KeyError occurred on this line:", e)
# unique_train_users = X_train['user_id'].unique()
# unique_test_users = X_test['user_id'].unique()

# print("Unique User IDs in X_train:", unique_train_users)
# print("Unique User IDs in X_test:", unique_test_users)

In [10]:
#defining entropy and information gain

In [11]:
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

In [12]:
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train.iloc[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train.iloc[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

def oob_score(tree, X_test, y_test):
    y_test = list(y_test)
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [13]:
import random
import numpy as np

def find_split_point(X_bootstrap, y_bootstrap, max_features):
    y_bootstrap = list(y_bootstrap)
    feature_ls = set()
    num_features = len(X_bootstrap[0])
    while len(feature_ls) < max_features:
        feature_idx = random.randint(0, num_features - 1)
        feature_ls.add(feature_idx)

    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        unique_values = np.unique(X_bootstrap[:, feature_idx])

        for split_point in unique_values:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}

            for i, value in enumerate(X_bootstrap[:, feature_idx]):
                if value <= split_point:
                    left_child['X_bootstrap'].append(X_bootstrap[i])
                    left_child['y_bootstrap'].append(y_bootstrap[i])
                else:
                    right_child['X_bootstrap'].append(X_bootstrap[i])
                    right_child['y_bootstrap'].append(y_bootstrap[i])

            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {
                    'information_gain': split_info_gain,
                    'left_child': left_child,
                    'right_child': right_child,
                    'split_point': split_point,
                    'feature_idx': feature_idx
                }

    return node


In [14]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [15]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

In [16]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']
    split_point = tree['split_point']

    print("Feature Index:", feature_idx)
    print("Split Point:", split_point)
    print("Test Value:", X_test[feature_idx])

    if X_test[feature_idx] <= split_point:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']


In [17]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)



In [ ]:
n_estimators = 20
max_features = 5
max_depth = 5
min_samples_split = 2

model = random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split)

In [ ]:
preds = predict_rf(model,X_test)
print(preds)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

In [ ]:
preds

In [ ]:
# preds

In [ ]:
# X_train.head()

In [ ]:
from sklearn.metrics import precision_score, f1_score, recall_score

# Calculate precision and F1-score with zero_division parameter
precision = precision_score(y_test, preds, zero_division=1)  # Set to 1 to avoid warnings
f1 = f1_score(y_test, preds, zero_division=1)
recall = recall_score(y_test, preds, zero_division=1)

In [ ]:
# precision

In [ ]:
# f1

In [ ]:
# recall

In [ ]:
# precision_class_0 = precision_score(y_test, preds, labels=[0], average=None)
# precision_class_0

In [ ]:
# precision_class_1 = precision_score(y_test, preds, labels=[1], average=None,zero_division=1)
# precision_class_1

In [ ]:
# # recall_class_1 = recall_score(y_test, preds, labels=[1], average=None,zero_division=1)
# recall_class_1

In [ ]:
# recall_class_0 = recall_score(y_test, preds, labels=[0], average=None,zero_division=1)
# recall_class_0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds, zero_division=1))